In [33]:
#from google.colab import drive
#drive.mount('/content/drive')

In [1]:
import pandas as pd
import re
import numpy as np
import math
import time
pd.options.display.max_columns = None

Initial Data Cleaning

In [2]:
# Assigning datatypes to columns based on initial data analysis
calendarDTypes = {"event_name_1": "category", 
                  "event_name_2": "category", 
                  "event_type_1": "category", 
                  "event_type_2": "category", 
                  "weekday": "category", 
                  'wm_yr_wk': 'int16', 
                  "wday": "int16",
                  "month": "int16", 
                  "year": "int16", 
                  "snap_CA": "float32", 
                  'snap_TX': 'float32', 
                  'snap_WI': 'float32' }

priceDTypes = {"store_id": "category", 
               "item_id": "category", 
               "wm_yr_wk": "int16",
               "sell_price":"float32"}


In [3]:
# Reading in calender and sell prices dataset
cal = pd.read_csv("calendar.csv", dtype = calendarDTypes)
prices = pd.read_csv("sell_prices.csv", dtype = priceDTypes)

In [4]:
firstDay = 1300
lastDay = 1941

# Use x sales days (columns) for training
numCols = [f"d_{day}" for day in range(firstDay, lastDay+1)]
catCols = ['id', 'item_id', 'dept_id','store_id', 'cat_id', 'state_id']
dtype = {numCol: "float32" for numCol in numCols} 
dtype.update({catCol: "category" for catCol in catCols if catCol != "id"})

# Read csv file
train_valid = pd.read_csv("sales_train_evaluation.csv", 
                 usecols = catCols + numCols, dtype = dtype)

In [5]:
train_valid.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1300,d_1301,d_1302,d_1303,d_1304,d_1305,d_1306,d_1307,d_1308,d_1309,d_1310,d_1311,d_1312,d_1313,d_1314,d_1315,d_1316,d_1317,d_1318,d_1319,d_1320,d_1321,d_1322,d_1323,d_1324,d_1325,d_1326,d_1327,d_1328,d_1329,d_1330,d_1331,d_1332,d_1333,d_1334,d_1335,d_1336,d_1337,d_1338,d_1339,d_1340,d_1341,d_1342,d_1343,d_1344,d_1345,d_1346,d_1347,d_1348,d_1349,d_1350,d_1351,d_1352,d_1353,d_1354,d_1355,d_1356,d_1357,d_1358,d_1359,d_1360,d_1361,d_1362,d_1363,d_1364,d_1365,d_1366,d_1367,d_1368,d_1369,d_1370,d_1371,d_1372,d_1373,d_1374,d_1375,d_1376,d_1377,d_1378,d_1379,d_1380,d_1381,d_1382,d_1383,d_1384,d_1385,d_1386,d_1387,d_1388,d_1389,d_1390,d_1391,d_1392,d_1393,d_1394,d_1395,d_1396,d_1397,d_1398,d_1399,d_1400,d_1401,d_1402,d_1403,d_1404,d_1405,d_1406,d_1407,d_1408,d_1409,d_1410,d_1411,d_1412,d_1413,d_1414,d_1415,d_1416,d_1417,d_1418,d_1419,d_1420,d_1421,d_1422,d_1423,d_1424,d_1425,d_1426,d_1427,d_1428,d_1429,d_1430,d_1431,d_1432,d_1433,d_1434,d_1435,d_1436,d_1437,d_1438,d_1439,d_1440,d_1441,d_1442,d_1443,d_1444,d_1445,d_1446,d_1447,d_1448,d_1449,d_1450,d_1451,d_1452,d_1453,d_1454,d_1455,d_1456,d_1457,d_1458,d_1459,d_1460,d_1461,d_1462,d_1463,d_1464,d_1465,d_1466,d_1467,d_1468,d_1469,d_1470,d_1471,d_1472,d_1473,d_1474,d_1475,d_1476,d_1477,d_1478,d_1479,d_1480,d_1481,d_1482,d_1483,d_1484,d_1485,d_1486,d_1487,d_1488,d_1489,d_1490,d_1491,d_1492,d_1493,d_1494,d_1495,d_1496,d_1497,d_1498,d_1499,d_1500,d_1501,d_1502,d_1503,d_1504,d_1505,d_1506,d_1507,d_1508,d_1509,d_1510,d_1511,d_1512,d_1513,d_1514,d_1515,d_1516,d_1517,d_1518,d_1519,d_1520,d_1521,d_1522,d_1523,d_1524,d_1525,d_1526,d_1527,d_1528,d_1529,d_1530,d_1531,d_1532,d_1533,d_1534,d_1535,d_1536,d_1537,d_1538,d_1539,d_1540,d_1541,d_1542,d_1543,d_1544,d_1545,d_1546,d_1547,d_1548,d_1549,d_1550,d_1551,d_1552,d_1553,d_1554,d_1555,d_1556,d_1557,d_1558,d_1559,d_1560,d_1561,d_1562,d_1563,d_1564,d_1565,d_1566,d_1567,d_1568,d_1569,d_1570,d_1571,d_1572,d_1573,d_1574,d_1575,d_1576,d_1577,d_1578,d_1579,d_1580,d_1581,d_1582,d_1583,d_1584,d_1585,d_1586,d_1587,d_1588,d_1589,d_1590,d_1591,d_1592,d_1593,d_1594,d_1595,d_1596,d_1597,d_1598,d_1599,d_1600,d_1601,d_1602,d_1603,d_1604,d_1605,d_1606,d_1607,d_1608,d_1609,d_1610,d_1611,d_1612,d_1613,d_1614,d_1615,d_1616,d_1617,d_1618,d_1619,d_1620,d_1621,d_1622,d_1623,d_1624,d_1625,d_1626,d_1627,d_1628,d_1629,d_1630,d_1631,d_1632,d_1633,d_1634,d_1635,d_1636,d_1637,d_1638,d_1639,d_1640,d_1641,d_1642,d_1643,d_1644,d_1645,d_1646,d_1647,d_1648,d_1649,d_1650,d_1651,d_1652,d_1653,d_1654,d_1655,d_1656,d_1657,d_1658,d_1659,d_1660,d_1661,d_1662,d_1663,d_1664,d_1665,d_1666,d_1667,d_1668,d_1669,d_1670,d_1671,d_1672,d_1673,d_1674,d_1675,d_1676,d_1677,d_1678,d_1679,d_1680,d_1681,d_1682,d_1683,d_1684,d_1685,d_1686,d_1687,d_1688,d_1689,d_1690,d_1691,d_1692,d_1693,d_1694,d_1695,d_1696,d_1697,d_1698,d_1699,d_1700,d_1701,d_1702,d_1703,d_1704,d_1705,d_1706,d_1707,d_1708,d_1709,d_1710,d_1711,d_1712,d_1713,d_1714,d_1715,d_1716,d_1717,d_1718,d_1719,d_1720,d_1721,d_1722,d_1723,d_1724,d_1725,d_1726,d_1727,d_1728,d_1729,d_1730,d_1731,d_1732,d_1733,d_1734,d_1735,d_1736,d_1737,d_1738,d_1739,d_1740,d_1741,d_1742,d_1743,d_1744,d_1745,d_1746,d_1747,d_1748,d_1749,d_1750,d_1751,d_1752,d_1753,d_1754,d_1755,d_1756,d_1757,d_1758,d_1759,d_1760,d_1761,d_1762,d_1763,d_1764,d_1765,d_1766,d_1767,d_1768,d_1769,d_1770,d_1771,d_1772,d_1773,d_1774,d_1775,d_1776,d_1777,d_1778,d_1779,d_1780,d_1781,d_1782,d_1783,d_1784,d_1785,d_1786,d_1787,d_1788,d_1789,d_1790,d_1791,d_1792,d_1793,d_1794,d_1795,d_1796,d_1797,d_1798,d_1799,d_1800,d_1801,d_1802,d_1803,d_1804,d_1805,d_1806,d_1807,d_1808,d_1809,d_1810,d_1811,d_1812,d_1813,d_1814,d_1815,d_1816,d_1817,d_1818,d_1819,d_1820,d_1821,d_1822,d_1823,d_1824,d_1825,d_1826,d_1827,d_1828,d_1829,d_1830,d_1831,d_1832,d_1833,d_1834,d_1835,d_1836,d_1837,d_1838,d_1839,d_1840,d_1841,d_1842,d_1843,d_1844,d_1845,d_1846,d_1847,d_1848,d_1849,d_1850,d_1851,d_1852,d_1853,d_1854,d_1855,d_1856,d_1857,d_1858,d_1859,d_1860,d_1861,d_1862,d_1863,d_1864,

In [6]:
# We need to convert the data from wide format to long format
stv = pd.melt(train_valid,
             id_vars = catCols,
             value_vars = [col for col in train_valid.columns if col.startswith("d_")],
             var_name = "d",
             value_name = "sales")

stv.head()

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,CA_1,HOBBIES,CA,d_1300,1.0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,CA_1,HOBBIES,CA,d_1300,0.0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,CA_1,HOBBIES,CA,d_1300,0.0
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,CA_1,HOBBIES,CA,d_1300,1.0
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,CA_1,HOBBIES,CA,d_1300,0.0


In [7]:
# Add placeholder for Validation and Testing Dataset
add_test = pd.DataFrame()
for i in range(1,29):
    temp_df = stv[catCols]
    temp_df = temp_df.drop_duplicates()
    temp_df['d'] = 'd_'+ str(lastDay+i)
    temp_df['sales'] = np.nan
    add_test = pd.concat([add_test,temp_df])

stv = pd.concat([stv,add_test])
stv = stv.reset_index(drop=True)

In [8]:
del temp_df, add_test

In [9]:
stv.shape

(20428300, 8)

In [10]:
# We need to merge the datasets to create one single dataset
stv = stv.merge(cal, on = "d")
stv = stv.merge(prices, on = ["store_id", "item_id", "wm_yr_wk"])

In [11]:
stv.tail()

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
20158405,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,WI_3,FOODS,WI,d_1969,NaN,2016-06-19,11621,Sunday,2,6,2016,NBAFinalsEnd,Sporting,Father's day,Cultural,0.0,0.0,0.0,3.98
20158406,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,WI_3,FOODS,WI,d_1968,NaN,2016-06-18,11621,Saturday,1,6,2016,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.28
20158407,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,WI_3,FOODS,WI,d_1969,NaN,2016-06-19,11621,Sunday,2,6,2016,NBAFinalsEnd,Sporting,Father's day,Cultural,0.0,0.0,0.0,1.28
20158408,FOODS_3_827_WI_3_evaluation,FOODS_3_827,FOODS_3,WI_3,FOODS,WI,d_1968,NaN,2016-06-18,11621,Saturday,1,6,2016,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.00
20158409,FOODS_3_827_WI_3_evaluation,FOODS_3_827,FOODS_3,WI_3,FOODS,WI,d_1969,NaN,2016-06-19,11621,Sunday,2,6,2016,NBAFinalsEnd,Sporting,Father's day,Cultural,0.0,0.0,0.0,1.00


In [12]:
# Function to reduce memory usage
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                       df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [13]:
# Calling function to reduce memory usage
stv = reduce_mem_usage(stv, verbose=True)

Mem. usage decreased to 1134.34 Mb (16.9% reduction)


## Feature Engineering

In [14]:
def merge_by_concat(df1, df2, merge_on):
    merged_gf = df1[merge_on]
    merged_gf = merged_gf.merge(df2, on=merge_on, how='left')
    new_columns = [col for col in list(merged_gf) if col not in merge_on]
    df1 = pd.concat([df1, merged_gf[new_columns]], axis=1)
    return df1

In [15]:
# Week on which the item was released
release_df = prices.groupby(['store_id','item_id'])['wm_yr_wk'].agg(['min']).reset_index()
release_df.columns = ['store_id','item_id','release']
release_df.head()

,store_id,item_id,release
0,CA_1,HOBBIES_1_001,11325
1,CA_1,HOBBIES_1_002,11121
2,CA_1,HOBBIES_1_003,11401
3,CA_1,HOBBIES_1_004,11106
4,CA_1,HOBBIES_1_005,11117


In [16]:
df_final = merge_by_concat(stv, release_df, ['store_id','item_id'])

In [17]:
df_final.head()

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,release
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,CA_1,HOBBIES,CA,d_1300,1.0,2014-08-20,11429,Wednesday,5,8,2014,NaN,NaN,NaN,NaN,0.0,0.0,0.0,8.257812,11325
1,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,CA_1,HOBBIES,CA,d_1301,0.0,2014-08-21,11429,Thursday,6,8,2014,NaN,NaN,NaN,NaN,0.0,0.0,0.0,8.257812,11325
2,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,CA_1,HOBBIES,CA,d_1302,0.0,2014-08-22,11429,Friday,7,8,2014,NaN,NaN,NaN,NaN,0.0,0.0,0.0,8.257812,11325
3,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,CA_1,HOBBIES,CA,d_1300,0.0,2014-08-20,11429,Wednesday,5,8,2014,NaN,NaN,NaN,NaN,0.0,0.0,0.0,3.970703,11121
4,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,CA_1,HOBBIES,CA,d_1301,0.0,2014-08-21,11429,Thursday,6,8,2014,NaN,NaN,NaN,NaN,0.0,0.0,0.0,3.970703,11121


In [18]:
df_final = df_final[df_final['wm_yr_wk']>=df_final['release']]
df_final = df_final.reset_index(drop=True)

In [19]:
#Keeping release as a feature
df_final['release'] = df_final['release'] - df_final['release'].min()
df_final['release'] = df_final['release'].astype(np.int16)

In [20]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20158410 entries, 0 to 20158409
Data columns (total 23 columns):
 #   Column        Dtype   
---  ------        -----   
 0   id            object  
 1   item_id       category
 2   dept_id       category
 3   store_id      category
 4   cat_id        category
 5   state_id      category
 6   d             object  
 7   sales         float16 
 8   date          object  
 9   wm_yr_wk      int16   
 10  weekday       category
 11  wday          int8    
 12  month         int8    
 13  year          int16   
 14  event_name_1  category
 15  event_type_1  category
 16  event_name_2  category
 17  event_type_2  category
 18  snap_CA       float16 
 19  snap_TX       float16 
 20  snap_WI       float16 
 21  sell_price    float16 
 22  release       int16   
dtypes: category(10), float16(5), int16(3), int8(2), object(3)
memory usage: 1019.0+ MB


In [21]:
df_final.shape

(20158410, 23)

In [22]:
# Merging Part 1
df_final.to_pickle('df_final_1.pkl')

In [23]:
df_final.head()

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,release
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,CA_1,HOBBIES,CA,d_1300,1.0,2014-08-20,11429,Wednesday,5,8,2014,NaN,NaN,NaN,NaN,0.0,0.0,0.0,8.257812,224
1,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,CA_1,HOBBIES,CA,d_1301,0.0,2014-08-21,11429,Thursday,6,8,2014,NaN,NaN,NaN,NaN,0.0,0.0,0.0,8.257812,224
2,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,CA_1,HOBBIES,CA,d_1302,0.0,2014-08-22,11429,Friday,7,8,2014,NaN,NaN,NaN,NaN,0.0,0.0,0.0,8.257812,224
3,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,CA_1,HOBBIES,CA,d_1300,0.0,2014-08-20,11429,Wednesday,5,8,2014,NaN,NaN,NaN,NaN,0.0,0.0,0.0,3.970703,20
4,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,CA_1,HOBBIES,CA,d_1301,0.0,2014-08-21,11429,Thursday,6,8,2014,NaN,NaN,NaN,NaN,0.0,0.0,0.0,3.970703,20


### Calendar Features

In [24]:
# Calendar Features
df_final['date'] = pd.to_datetime(df_final['date'])
df_final['day'] = df_final['date'].dt.day.astype(np.int8)
df_final['week'] = df_final['date'].dt.week.astype(np.int8)
df_final['month'] = df_final['date'].dt.month.astype(np.int8)
df_final['year'] = df_final['date'].dt.year
df_final['year_enc'] = (df_final['year'] - df_final['year'].min()).astype(np.int8)
df_final['week_month'] = df_final['day'].apply(lambda x: math.ceil(x/7)).astype(np.int8)
df_final['day_week'] = df_final['date'].dt.dayofweek.astype(np.int8)
df_final['weekend'] = (df_final['day_week']>=5).astype(np.int8)

C:\Users\aravi\AppData\Local\Temp\ipykernel_68252\3291639626.py:4: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df_final['week'] = df_final['date'].dt.week.astype(np.int8)


### Price Features

In [25]:
# Price Features
df_final['price_max'] = df_final.groupby(['store_id','item_id'])['sell_price'].transform('max')
df_final['price_min'] = df_final.groupby(['store_id','item_id'])['sell_price'].transform('min')
df_final['price_std'] = df_final.groupby(['store_id','item_id'])['sell_price'].transform('std')
df_final['price_mean'] = df_final.groupby(['store_id','item_id'])['sell_price'].transform('mean')
df_final['price_norm'] = df_final['sell_price']/df_final['price_max']

In [26]:
# Momentum based features
df_final['price_momentum'] = df_final['sell_price']/df_final.groupby(['store_id','item_id'])['sell_price'] \
                                        .transform(lambda x: x.shift(1))
df_final['price_momentum_m'] = df_final['sell_price']/df_final.groupby(['store_id','item_id','month'])['sell_price'] \
                                        .transform('mean')
df_final['price_momentum_y'] = df_final['sell_price']/df_final.groupby(['store_id','item_id','year_enc'])['sell_price'] \
                                        .transform('mean')

In [37]:
df_final.head()

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,release,day,week,year_enc,week_month,day_week,weekend,price_max,price_min,price_std,price_mean,price_norm,price_momentum,price_momentum_m,price_momentum_y
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,CA_1,HOBBIES,CA,d_1300,1.0,2014-08-20,11429,Wednesday,5,8,2014,NaN,NaN,NaN,NaN,0.0,0.0,0.0,8.257812,224,20,34,0,3,2,0,8.382812,8.257812,0.038741,8.273438,0.984863,NaN,1.0,1.0
1,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,CA_1,HOBBIES,CA,d_1301,0.0,2014-08-21,11429,Thursday,6,8,2014,NaN,NaN,NaN,NaN,0.0,0.0,0.0,8.257812,224,21,34,0,3,3,0,8.382812,8.257812,0.038741,8.273438,0.984863,1.0,1.0,1.0
2,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,CA_1,HOBBIES,CA,d_1302,0.0,2014-08-22,11429,Friday,7,8,2014,NaN,NaN,NaN,NaN,0.0,0.0,0.0,8.257812,224,22,34,0,4,4,0,8.382812,8.257812,0.038741,8.273438,0.984863,1.0,1.0,1.0
3,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,CA_1,HOBBIES,CA,d_1300,0.0,2014-08-20,11429,Wednesday,5,8,2014,NaN,NaN,NaN,NaN,0.0,0.0,0.0,3.970703,20,20,34,0,3,2,0,3.970703,3.970703,0.000000,3.970703,1.000000,NaN,1.0,1.0
4,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,CA_1,HOBBIES,CA,d_1301,0.0,2014-08-21,11429,Thursday,6,8,2014,NaN,NaN,NaN,NaN,0.0,0.0,0.0,3.970703,20,21,34,0,3,3,0,3.970703,3.970703,0.000000,3.970703,1.000000,1.0,1.0,1.0


In [27]:
df_final = reduce_mem_usage(df_final, verbose=True)

Mem. usage decreased to 1441.93 Mb (13.8% reduction)


In [29]:
# Converting day column 'd' to int
df_final['d'] = df_final['d'].apply(lambda x: x[2:]).astype(np.int16)

In [30]:
del df_final['date']

In [31]:
# Dropping weekday and wm_yr_wk
df_final.drop(['wm_yr_wk','weekday', 'year'], axis=1, inplace =True)

In [32]:
df_final.drop(['wday'], axis=1, inplace =True)

In [34]:
df_final.to_pickle('df_price_cal_feat.pkl')

### Lag Features

In [35]:
df_final.head()

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,month,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,release,day,week,year_enc,week_month,day_week,weekend,price_max,price_min,price_std,price_mean,price_norm,price_momentum,price_momentum_m,price_momentum_y
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,CA_1,HOBBIES,CA,1300,1.0,8,NaN,NaN,NaN,NaN,0.0,0.0,0.0,8.257812,224,20,34,0,3,2,0,8.382812,8.257812,0.038727,8.273438,0.984863,NaN,1.0,1.0
1,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,CA_1,HOBBIES,CA,1301,0.0,8,NaN,NaN,NaN,NaN,0.0,0.0,0.0,8.257812,224,21,34,0,3,3,0,8.382812,8.257812,0.038727,8.273438,0.984863,1.0,1.0,1.0
2,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,CA_1,HOBBIES,CA,1302,0.0,8,NaN,NaN,NaN,NaN,0.0,0.0,0.0,8.257812,224,22,34,0,4,4,0,8.382812,8.257812,0.038727,8.273438,0.984863,1.0,1.0,1.0
3,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,CA_1,HOBBIES,CA,1300,0.0,8,NaN,NaN,NaN,NaN,0.0,0.0,0.0,3.970703,20,20,34,0,3,2,0,3.970703,3.970703,0.000000,3.970703,1.000000,NaN,1.0,1.0
4,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,CA_1,HOBBIES,CA,1301,0.0,8,NaN,NaN,NaN,NaN,0.0,0.0,0.0,3.970703,20,21,34,0,3,3,0,3.970703,3.970703,0.000000,3.970703,1.000000,1.0,1.0,1.0


In [37]:
df_lag = df_final[['id','d','sales']]
shift = 28
target = 'sales'

# Creating lag days with 28 days to 42 days lag
lag_days = [col for col in range(shift,shift + 15)]
df_lag = df_lag.assign(**{
        '{}_lag_{}'.format(col, l): df_lag.groupby(['id'])[col].transform(lambda x: x.shift(l))
        for l in lag_days
        for col in [target]
    })

for col in list(df_lag):
    if 'lag' in col:
        df_lag[col] = df_lag[col].astype(np.float16)



Create lags
3.49 min: Lags


In [38]:
# Creating rolling means and rolling standard deviation starting with 28 day lag and then for 7,14,30,60 and 180 days

for i in [7,14,30,60,180]:
    print('Rolling period:', i)
    df_lag['rolling_mean_'+ str(i)] = df_lag.groupby(['id'])[target].transform(lambda x: x.shift(shift).rolling(i).mean()).astype(np.float16)
    df_lag['rolling_std_'+ str(i)]  = df_lag.groupby(['id'])[target].transform(lambda x: x.shift(shift).rolling(i).std()).astype(np.float16)

Create rolling aggs
Rolling period: 7
Rolling period: 14
Rolling period: 30
Rolling period: 60
Rolling period: 180
3.37 min: Lags


In [7]:
# Rollings with sliding shift
# start_time = time.time()
# for d_shift in [1,7,14]: 
#     print('Shifting period:', d_shift)
#     for d_window in [7,14,30,60]:
#         col_name = 'rolling_mean_tmp_'+str(d_shift)+'_'+str(d_window)
#         df_lag[col_name] = df_lag.groupby(['id'])[TARGET].transform(lambda x: x.shift(d_shift).rolling(d_window).mean()).astype(np.float16)
    
    
# print('%0.2f min: Lags' % ((time.time() - start_time) / 60))

Shifting period: 1
Shifting period: 7
Shifting period: 14
4.76 min: Lags


In [39]:
df_lag.shape

(20158410, 28)

In [40]:
df_lag.to_pickle('df_lags.pkl')

### Additional Features

In [41]:
# Encoded features 
df_enc = df_final
df_enc['sales'][df_enc['d']>(1969-28)] = np.nan
base_cols = list(df_enc)

icols =  [
            ['item_id', 'state_id'],
            ['item_id', 'store_id']
            ]

for col in icols:
    print('Encoding', col)
    col_name = '_'+'_'.join(col)+'_'
    df_enc['enc'+col_name+'mean'] = df_enc.groupby(col)['sales'].transform('mean').astype(np.float16)
    df_enc['enc'+col_name+'std'] = df_enc.groupby(col)['sales'].transform('std').astype(np.float16)

keep_cols = [col for col in list(df_enc) if col not in base_cols]
df_enc = df_enc[['id','d']+keep_cols]

C:\Users\aravi\AppData\Local\Temp\ipykernel_68252\2309401774.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_enc['sales'][df_enc['d']>(1969-28)] = np.nan


Encoding ['item_id', 'state_id']
Encoding ['item_id', 'store_id']


In [42]:
df_final.head()

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,month,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,release,day,week,year_enc,week_month,day_week,weekend,price_max,price_min,price_std,price_mean,price_norm,price_momentum,price_momentum_m,price_momentum_y,enc_item_id_state_id_mean,enc_item_id_state_id_std,enc_item_id_store_id_mean,enc_item_id_store_id_std
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,CA_1,HOBBIES,CA,1300,1.0,8,NaN,NaN,NaN,NaN,0.0,0.0,0.0,8.257812,224,20,34,0,3,2,0,8.382812,8.257812,0.038727,8.273438,0.984863,NaN,1.0,1.0,0.670410,0.941406,0.699219,0.953613
1,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,CA_1,HOBBIES,CA,1301,0.0,8,NaN,NaN,NaN,NaN,0.0,0.0,0.0,8.257812,224,21,34,0,3,3,0,8.382812,8.257812,0.038727,8.273438,0.984863,1.0,1.0,1.0,0.670410,0.941406,0.699219,0.953613
2,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,CA_1,HOBBIES,CA,1302,0.0,8,NaN,NaN,NaN,NaN,0.0,0.0,0.0,8.257812,224,22,34,0,4,4,0,8.382812,8.257812,0.038727,8.273438,0.984863,1.0,1.0,1.0,0.670410,0.941406,0.699219,0.953613
3,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,CA_1,HOBBIES,CA,1300,0.0,8,NaN,NaN,NaN,NaN,0.0,0.0,0.0,3.970703,20,20,34,0,3,2,0,3.970703,3.970703,0.000000,3.970703,1.000000,NaN,1.0,1.0,0.220459,0.535156,0.317871,0.660156
4,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,CA_1,HOBBIES,CA,1301,0.0,8,NaN,NaN,NaN,NaN,0.0,0.0,0.0,3.970703,20,21,34,0,3,3,0,3.970703,3.970703,0.000000,3.970703,1.000000,1.0,1.0,1.0,0.220459,0.535156,0.317871,0.660156


In [43]:
df_enc.head()

,id,d,enc_item_id_state_id_mean,enc_item_id_state_id_std,enc_item_id_store_id_mean,enc_item_id_store_id_std
0,HOBBIES_1_001_CA_1_evaluation,1300,0.670410,0.941406,0.699219,0.953613
1,HOBBIES_1_001_CA_1_evaluation,1301,0.670410,0.941406,0.699219,0.953613
2,HOBBIES_1_001_CA_1_evaluation,1302,0.670410,0.941406,0.699219,0.953613
3,HOBBIES_1_002_CA_1_evaluation,1300,0.220459,0.535156,0.317871,0.660156
4,HOBBIES_1_002_CA_1_evaluation,1301,0.220459,0.535156,0.317871,0.660156


In [45]:
df_enc.to_pickle('grouped_enc.pkl')

In [44]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20158410 entries, 0 to 20158409
Data columns (total 36 columns):
 #   Column                     Dtype   
---  ------                     -----   
 0   id                         object  
 1   item_id                    category
 2   dept_id                    category
 3   store_id                   category
 4   cat_id                     category
 5   state_id                   category
 6   d                          int16   
 7   sales                      float16 
 8   month                      int8    
 9   event_name_1               category
 10  event_type_1               category
 11  event_name_2               category
 12  event_type_2               category
 13  snap_CA                    float16 
 14  snap_TX                    float16 
 15  snap_WI                    float16 
 16  sell_price                 float16 
 17  release                    int16   
 18  day                        int8    
 19  week               

In [47]:
df_lag.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20158410 entries, 0 to 20158409
Data columns (total 28 columns):
 #   Column            Dtype  
---  ------            -----  
 0   id                object 
 1   d                 int16  
 2   sales             float16
 3   sales_lag_28      float16
 4   sales_lag_29      float16
 5   sales_lag_30      float16
 6   sales_lag_31      float16
 7   sales_lag_32      float16
 8   sales_lag_33      float16
 9   sales_lag_34      float16
 10  sales_lag_35      float16
 11  sales_lag_36      float16
 12  sales_lag_37      float16
 13  sales_lag_38      float16
 14  sales_lag_39      float16
 15  sales_lag_40      float16
 16  sales_lag_41      float16
 17  sales_lag_42      float16
 18  rolling_mean_7    float16
 19  rolling_std_7     float16
 20  rolling_mean_14   float16
 21  rolling_std_14    float16
 22  rolling_mean_30   float16
 23  rolling_std_30    float16
 24  rolling_mean_60   float16
 25  rolling_std_60    float16
 26  rolling_mean

In [ ]:
df_enc.shape

In [48]:
catcols = df_final.select_dtypes(include='category').columns

In [49]:
#Encoding the categorical columns
for col in catcols:
    df_final[col] = df_final[col].cat.codes.astype("int16")
    df_final[col] -= df_final[col].min()

In [50]:
del prices, cal

In [51]:
grid_df = pd.concat([df_final, df_lag.iloc[:,3:]], axis=1)

In [52]:
grid_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20158410 entries, 0 to 20158409
Data columns (total 61 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   id                         object 
 1   item_id                    int16  
 2   dept_id                    int16  
 3   store_id                   int16  
 4   cat_id                     int16  
 5   state_id                   int16  
 6   d                          int16  
 7   sales                      float16
 8   month                      int8   
 9   event_name_1               int16  
 10  event_type_1               int16  
 11  event_name_2               int16  
 12  event_type_2               int16  
 13  snap_CA                    float16
 14  snap_TX                    float16
 15  snap_WI                    float16
 16  sell_price                 float16
 17  release                    int16  
 18  day                        int8   
 19  week                       int8   
 20  

In [46]:
grid_df.to_pickle('grid_df.pkl')

In [54]:
del df_final

In [ ]:
reduce_mem_usage(df_final, verbose=True)